In [1]:
# Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

In [11]:
# Load the data
df=pd.read_csv('/content/drive/MyDrive/assignments-BE/ANN/creditcard.csv')
df.head()
df['Class'].value_counts()# highly imbalanced data
# According to the data mentioned in kaggle, input features names are omitted,so we can't use domain knowledge to omit few features, we go with EDA alone.

,count
Class,
0,284315
1,492


In [3]:
# EDA
# scaling 'Time and amount features as these aren't scaled according to the dataset
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df['scaled_amount']=scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time']=scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,scaled_amount,scaled_time
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.244964,-1.996583
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,-0.342475,-1.996583
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1.160686,-1.996562
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.140534,-1.996562
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,-0.073403,-1.996541


In [4]:
# Separating input and output
X=df.drop(['Time','Amount','Class'],axis=1)
y=df['Class']


In [5]:
# Traintest split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=5)

In [7]:
# model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Dense(32,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [8]:
# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
# Training the model
history=model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.2,verbose=1)

Epoch 1/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9994 - loss: 0.0029 - val_accuracy: 0.9995 - val_loss: 0.0022
Epoch 2/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9994 - loss: 0.0027 - val_accuracy: 0.9994 - val_loss: 0.0025
Epoch 3/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9994 - val_loss: 0.0023
Epoch 4/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 0.9994 - val_loss: 0.0025
Epoch 5/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - accuracy: 0.9995 - loss: 0.0022 - val_accuracy: 0.9994 - val_loss: 0.0025
Epoch 6/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9994 - val_loss: 0.0025
Epoch 7/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9996 - loss: 0.0017 - val_accuracy: 0.9992 - val_loss: 0.0029
Epoch 8/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9996 - loss: 0

In [18]:
# Testing the model
loss,accuracy=model.evaluate(X_test,y_test)
y_pred=model.predict(X_test)
y_pred_labels=(y_pred>0.5).astype(int)# as y_pred gives probabilities
print('test loss=',loss)
print('test accuracy=',accuracy)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9996 - loss: 0.0029
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
test loss= 0.004128847271203995
test accuracy= 0.9994206428527832


In [25]:
print('Confusion matrix=',tf.math.confusion_matrix(y_test,y_pred_labels))

Confusion matrix= tf.Tensor(
[[56855     3]
 [   33    71]], shape=(2, 2), dtype=int32)


In [27]:
# precision, recall, f1 score
from sklearn.metrics import precision_score,recall_score,f1_score
print('precision=',precision_score(y_test,y_pred_labels))
print('recall=',recall_score(y_test,y_pred_labels))
print('f1 score=',f1_score(y_test,y_pred_labels))

precision= 0.9594594594594594
recall= 0.6826923076923077
f1 score= 0.797752808988764


In [ ]:
# recall is very poor

In [33]:
# Testing the model
loss,accuracy=model.evaluate(X_test,y_test)
y_pred=model.predict(X_test)
y_pred_labels=(y_pred>0.3).astype(int)# as y_pred gives probabilities
print('test loss=',loss)
print('test accuracy=',accuracy)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9996 - loss: 0.0029
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
test loss= 0.004128847271203995
test accuracy= 0.9994206428527832


In [34]:
print('Confusion matrix=',tf.math.confusion_matrix(y_test,y_pred_labels))


Confusion matrix= tf.Tensor(
[[56848    10]
 [   20    84]], shape=(2, 2), dtype=int32)


In [36]:
print('precision=',precision_score(y_test,y_pred_labels))
print('recall=',recall_score(y_test,y_pred_labels))
print('f1 score=',f1_score(y_test,y_pred_labels))

precision= 0.8936170212765957
recall= 0.8076923076923077
f1 score= 0.8484848484848485


In [ ]:
# here recall is improved by decreasing y_pred to 0.3,FN decreased

In [37]:

# accuracy is not taken as consideration as data is highly imbalanced